In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from dateutil import tz
from tqdm import tqdm
from glob import glob
from collections import Counter
from plotly.offline import iplot
from IPython.core.interactiveshell import InteractiveShell


InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline

In [2]:
list_files = sorted(glob('./data/*.csv'))
list_files

['./data/bangkok_pm25.csv',
 './data/patumwan_weather.csv',
 './data/training.csv']

In [ ]:
# http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID=50t&param=PM25&type=hr&sdate=2016-10-01&edate=2019-01-17&stime=00&etime=24
# https://api.wunderground.com/

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import pandas as pd
from datetime import timedelta, date
from tqdm import tqdm

start_date = date(2016, 3, 3)
end_date = date(2019, 1, 28)


def daterange(start_date, end_date):
    for n in tqdm(range(int ((end_date - start_date).days))):
        yield start_date + timedelta(n)

write_header = 1
write_header_file = 1 # change to 0 to continue without writing header to csv
for single_date in daterange(start_date, end_date):
    df_daily = pd.DataFrame()
    ymd = single_date.strftime("%Y%m%d")
    url_test = "https://api-ak.wunderground.com/api/606f3f6977348613/history_"+ymd+"null/units:metric/v:2.0/q/pws:IBANGKOK26.json"
    req = requests.get(url_test)
    js = json.loads(req.text)
    if js['history']['days'] == []:
        print("empty data on",ymd)
        continue
    for ob in js['history']['days'][0]['observations']:
        ob['date'] = ob['date']['iso8601']
        df_row = pd.DataFrame(ob, index=[0]) 
        df_row.index = df_row['date']
        df_row = df_row.drop(columns=['date'])
        if write_header == 1:
            df_daily = df_row
            write_header = 0
        else:
            df_daily = df_daily.append(df_row)

    with open('patumwan_weather.csv','a+') as fd:
        fd.write(df_daily.to_csv(header=write_header_file))
        write_header_file = 0

In [3]:
def convert(x):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
#     print(x)
    return x.replace(tzinfo=from_zone).astimezone(to_zone)

In [7]:
pm25_df = pd.read_csv(list_files[0])
pm25_df['date'] = pm25_df['Year'].astype(str) + '-' + pm25_df['Month'].astype(str).str.zfill(2) + '-' + pm25_df['Day'].astype(str).str.zfill(2) + ' ' + pm25_df['UTC Hour'].astype(str).str.zfill(2)
pm25_df['date'] = pd.to_datetime(pm25_df['date'], format='%Y-%m-%d %H')

# convert time zone
pm25_df.head(20)
pm25_df['date'] = pm25_df['date'].apply(convert)
pm25_df['Year'] = pm25_df['date'].dt.year
pm25_df['Month'] = pm25_df['date'].dt.month
pm25_df['Day'] = pm25_df['date'].dt.day
pm25_df['UTC Hour'] = pm25_df['date'].dt.hour

pm25_df['day_of_week'] = pm25_df['date'].dt.day_name()
pm25_df['date'] = pm25_df['date'].dt.strftime('%Y-%m-%d %H')
dateinfo = pm25_df[['date', 'Year', 'Month', 'Day', 'UTC Hour']]
pm25_df = pm25_df.drop(['Year', 'Month', 'Day', 'UTC Hour', 'Retrospective', 'PM10_mask'], axis=1)

pm25_df.info()
pm25_df.head(20)

,Year,Month,Day,UTC Hour,PM2.5,PM10_mask,Retrospective,date
0,2016,3,3,8,62.9,1,0,2016-03-03 08:00:00
1,2016,3,3,9,62.9,1,0,2016-03-03 09:00:00
2,2016,3,3,10,55.5,1,0,2016-03-03 10:00:00
3,2016,3,3,11,55.5,1,0,2016-03-03 11:00:00
4,2016,3,3,12,47.9,1,0,2016-03-03 12:00:00
5,2016,3,3,13,43.6,1,0,2016-03-03 13:00:00
6,2016,3,3,14,28.6,1,0,2016-03-03 14:00:00
7,2016,3,3,15,33.6,1,0,2016-03-03 15:00:00
8,2016,3,3,16,34.8,1,0,2016-03-03 16:00:00
9,2016,3,3,17,31.3,1,0,2016-03-03 17:00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23338 entries, 0 to 23337
Data columns (total 3 columns):
PM2.5          23338 non-null float64
date           23338 non-null object
day_of_week    23338 non-null object
dtypes: float64(1), object(2)
memory usage: 547.1+ KB


,PM2.5,date,day_of_week
0,62.9,2016-03-03 15,Thursday
1,62.9,2016-03-03 16,Thursday
2,55.5,2016-03-03 17,Thursday
3,55.5,2016-03-03 18,Thursday
4,47.9,2016-03-03 19,Thursday
5,43.6,2016-03-03 20,Thursday
6,28.6,2016-03-03 21,Thursday
7,33.6,2016-03-03 22,Thursday
8,34.8,2016-03-03 23,Thursday
9,31.3,2016-03-04 00,Friday


In [8]:
weather_df = pd.read_csv(list_files[1])
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df['date'] = weather_df.date.dt.strftime('%Y-%m-%d %H')
weather_df.info()
weather_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245497 entries, 0 to 245496
Data columns (total 41 columns):
date                  245497 non-null object
temperature           245491 non-null float64
dewpoint              245491 non-null float64
humidity              245492 non-null float64
wind_speed            245492 non-null float64
wind_gust_speed       228129 non-null float64
wind_dir_degrees      244164 non-null float64
wind_dir              244273 non-null object
pressure              245497 non-null float64
windchill             0 non-null float64
heatindex             207504 non-null float64
precip                244511 non-null float64
precip_rate           245497 non-null float64
precip_1hr            245497 non-null float64
precip_today          245210 non-null float64
solarradiation        245491 non-null float64
uv_index              245316 non-null float64
temperature_indoor    9150 non-null float64
humidity_indoor       9150 non-null float64
software_type         2454

,date,temperature,dewpoint,humidity,wind_speed,wind_gust_speed,wind_dir_degrees,wind_dir,pressure,windchill,...,aqnoy,aqno3,aqso4,aqso2,aqso2t,aqco,aqcot,aqec,aqoc,aqbc
0,2016-03-02 17,27.9,19.4,60.0,0.0,0.0,117.0,ESE,1011.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-03-02 17,27.9,20.0,62.0,0.0,0.0,117.0,ESE,1011.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-03-02 17,28.1,20.0,62.0,3.2,4.8,117.0,ESE,1011.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-03-02 17,28.1,20.6,63.0,1.6,6.4,117.0,ESE,1011.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-03-02 18,27.8,20.0,63.0,0.0,0.0,117.0,ESE,1010.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = pd.merge(pm25_df, weather_df, on='date')

In [10]:
df.head()

,PM2.5,date,day_of_week,temperature,dewpoint,humidity,wind_speed,wind_gust_speed,wind_dir_degrees,wind_dir,...,aqnoy,aqno3,aqso4,aqso2,aqso2t,aqco,aqcot,aqec,aqoc,aqbc
0,62.9,2016-03-03 15,Thursday,28.6,21.7,66.0,0.0,14.5,211.0,SSW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62.9,2016-03-03 15,Thursday,28.3,22.2,69.0,0.0,6.4,100.0,East,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62.9,2016-03-03 15,Thursday,28.2,22.2,69.0,3.2,8.0,237.0,WSW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62.9,2016-03-03 16,Thursday,28.1,22.2,70.0,0.0,1.6,333.0,NNW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62.9,2016-03-03 16,Thursday,28.0,22.8,72.0,1.6,6.4,69.0,ENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
len(df)

227213

In [12]:
col = []
for c, v in zip(df.columns, df.isna().sum() < 10):
    if v == True:
        col.append(c)

In [13]:
df = df[col]

In [14]:
df = df.drop('software_type', axis=1)

In [15]:
tmp = df.groupby(['date']).agg(['mean', 'median', 'std', 'min', 'max']).reset_index()
tmp.columns = tmp.columns.map('|'.join)
tmp.head()
# df = df.groupby(['date']).agg(['mean', 'median']).reset_index()

,date|,PM2.5|mean,PM2.5|median,PM2.5|std,PM2.5|min,PM2.5|max,temperature|mean,temperature|median,temperature|std,temperature|min,...,precip_1hr|mean,precip_1hr|median,precip_1hr|std,precip_1hr|min,precip_1hr|max,solarradiation|mean,solarradiation|median,solarradiation|std,solarradiation|min,solarradiation|max
0,2016-03-03 15,62.9,62.9,0.0,62.9,62.9,28.366667,28.30,0.208167,28.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-03-03 16,62.9,62.9,0.0,62.9,62.9,28.000000,28.00,0.081650,27.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-03-03 17,55.5,55.5,0.0,55.5,55.5,27.700000,27.70,0.081650,27.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-03-03 18,55.5,55.5,0.0,55.5,55.5,27.433333,27.40,0.057735,27.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-03-03 19,47.9,47.9,0.0,47.9,47.9,27.250000,27.25,0.057735,27.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
tmp = tmp.rename(columns={"date|": 'date'})

In [17]:
tmp['date'] = pd.to_datetime(tmp['date'], format='%Y-%m-%d %H')
tmp['day_of_week'] = tmp['date'].dt.day_name()
dateinfo['date'] = pd.to_datetime(dateinfo['date'], format='%Y-%m-%d %H')
tmp = pd.merge(tmp, dateinfo, on='date')
tmp = tmp.rename(columns={"UTC Hour": "Hour"})

In [18]:
tmp['day_of_year'] = tmp['date'].dt.dayofyear
tmp['quarter'] = tmp['date'].dt.quarter
tmp['week_of_year'] = tmp['date'].dt.weekofyear

In [19]:
tmp.to_csv('./data/training.csv', index=False)

In [20]:
tmp.head()

,date,PM2.5|mean,PM2.5|median,PM2.5|std,PM2.5|min,PM2.5|max,temperature|mean,temperature|median,temperature|std,temperature|min,...,solarradiation|min,solarradiation|max,day_of_week,Year,Month,Day,Hour,day_of_year,quarter,week_of_year
0,2016-03-03 15:00:00,62.9,62.9,0.0,62.9,62.9,28.366667,28.30,0.208167,28.2,...,0.0,0.0,Thursday,2016,3,3,15,63,1,9
1,2016-03-03 16:00:00,62.9,62.9,0.0,62.9,62.9,28.000000,28.00,0.081650,27.9,...,0.0,0.0,Thursday,2016,3,3,16,63,1,9
2,2016-03-03 17:00:00,55.5,55.5,0.0,55.5,55.5,27.700000,27.70,0.081650,27.6,...,0.0,0.0,Thursday,2016,3,3,17,63,1,9
3,2016-03-03 18:00:00,55.5,55.5,0.0,55.5,55.5,27.433333,27.40,0.057735,27.4,...,0.0,0.0,Thursday,2016,3,3,18,63,1,9
4,2016-03-03 19:00:00,47.9,47.9,0.0,47.9,47.9,27.250000,27.25,0.057735,27.2,...,0.0,0.0,Thursday,2016,3,3,19,63,1,9
